In [109]:
### 
import plotly.graph_objects as go # Note: Requires Version 4.14.3 to run without dedacted errors
import numpy as np # Note: Requires Version: 1.21.5
from ipywidgets import widgets, FileUpload, VBox # Note: Requires Version: 7.5.1 to run properly
import os
from IPython.display import display, clear_output # Note: Requires Version 7.34.0 to run properly 
import pickle
import json

X = widgets.IntText(description='X:', value=50)
Y = widgets.IntText(description='Y:', value=75)
submit_dim_button = widgets.Button(description="Submit X & Y dim")
display(X, Y, submit_dim_button)

# dictionary for saved clicked points
gallery = {}


def on_submit_dim_button(_):
    x, y = np.meshgrid(np.arange(X.value), np.arange(Y.value))
    f = go.FigureWidget([go.Scatter(x=x.flatten(), y=y.flatten(), mode='markers')])
    scatter = f.data[0]
    colors = ['#B2B4B8'] * (X.value * Y.value)
    scatter.marker.color = colors
    scatter.marker.size = [8] * (X.value * Y.value)
    scatter.hovertext = [f"Point {i}<br>X: {x.flatten()[i]}<br>Y: {y.flatten()[i]}" for i in range(len(x.flatten()))]
    f.layout.hovermode = 'closest'
    


    # create our callback function
    def update_point(trace, points, selector):
        c = list(scatter.marker.color)
        s = list(scatter.marker.size)

        # create our input widgets
        section_input = widgets.Text(description='Section:', value="", continuous_update=False)
        row_input = widgets.Text(description='Row:', value="", continuous_update=False)
        seat_input = widgets.Text(description='Seat:', value="", continuous_update=False)

        # create a button widget to submit the input values
        submit_button = widgets.Button(description="Submit")

        # create a VBox container for the input widgets and the submit button
        inputs_container = widgets.HBox([section_input, row_input, seat_input, submit_button])
        

        for i in points.point_inds:
            x_val = x.flatten()[i]
            y_val = y.flatten()[i]

            key_name = None
            for seat, seat_data in gallery.items():
                if seat_data['x'] == x_val and seat_data['y'] == y_val:
                    key_name = seat
                    break

            if key_name is not None:
                # Remove the seat from the gallery
                del gallery[key_name]

            else:
                c[i] = '#010C80'
                s[i] = 8

                # display the input widgets and the submit button
                display(inputs_container)

                def on_submit(_):
                    section = section_input.value.strip()
                    row = row_input.value.strip()
                    seat = seat_input.value.strip()
                    key_name = ""
                    if section:
                        key_name += f"Sec {section} "
                    if row:
                        key_name += f"Row {row} "
                    if seat:
                        key_name += f"Seat {seat}"
                    if key_name:
                        gallery[key_name] = {"name": key_name, "sec": section, "row": row, "seat number": seat, "x": x_val, "y": y_val, "color": '#010C80', "minted": False, "tokenID": None, "price": 0, "bought": False}
                        if f'Point {i}' in gallery:
                            del gallery[f'Point {i}']

                        with f.batch_update():
                            scatter.marker.color = c
                            scatter.marker.size = s

                    # hide the input widgets and the submit button
                    inputs_container.close()
                    

                submit_button.on_click(on_submit)
                
                
                # define a function to handle the keydown event for input widgets
                def on_keydown(change):
                    if change.new == '':
                        return
                    if change['name'] == 'value' and change['type'] == 'change':
                        if change['new'] == '\n':
                            on_submit(None)
                            return
                        if change['owner'] == section_input and row_input.value and seat_input.value:
                            on_submit(None)
                            return
                        if change['owner'] == row_input and section_input.value and seat_input.value:
                            on_submit(None)
                            return
                        if change['owner'] == seat_input and section_input.value and row_input.value:
                            on_submit(None)
                            return

                # add event listener for the "keydown" event to each input widget
                section_input.observe(on_keydown)
                row_input.observe(on_keydown)
                seat_input.observe(on_keydown)

    scatter.on_click(update_point)

    ##########

    # function & associated button/button functions that executes an update of all scatter.marker points
    def update_hovertext():
        new_hovertext = []
        new_colors = []
        for i in range(len(x.flatten())):
            x_val = x.flatten()[i]
            y_val = y.flatten()[i]
            found_in_gallery = False
            for key_name, point_info in gallery.items():
                if point_info['x'] == x_val and point_info['y'] == y_val:
                    new_hovertext.append(f"name: {point_info['name']}<br>sec: {point_info['sec']}<br>row: {point_info['row']}<br>seat number: {point_info['seat number']}<br>X: {x_val}<br>Y: {y_val}")
                    new_colors.append(point_info['color'])
                    found_in_gallery = True
                    break
            if not found_in_gallery:
                new_hovertext.append(f"Point {i}<br>X: {x_val}<br>Y: {y_val}")
                new_colors.append('#B2B4B8')


        scatter.hovertext = tuple(new_hovertext)
        scatter.marker.color = tuple(new_colors)

    update_hovertext_button = widgets.Button(description="Update Hovertext")

    def on_button_click(update_hovertext_button):
        update_hovertext()

    update_hovertext_button.on_click(on_button_click)

    ##########


    xmax = np.max(x) + 25
    xmin = np.min(x) - 25
    ymax = np.max(y) + 25
    ymin = np.min(y) - 25



    f.update_layout(xaxis_range=[xmin, xmax], yaxis_range=[ymin, ymax], xaxis_autorange=True, yaxis_autorange=True)
    f.update_layout(
        title='Venue Builder - Section Constructor',
        xaxis_title='Aisle',
        yaxis_title='Row'
    )
    
    ### test
    f.update_layout(
    width=800,
    height=800
    )
    ### test

    f



    def view_gallery(): 
        return gallery

    view_gallery_output = widgets.Output()

    def on_view_gallery_button_click(view_gallery_button):
        with view_gallery_output:
            display(view_gallery())
            clear_output(wait=True)

    view_gallery_button = widgets.Button(description="View Gallery")
    view_gallery_button.on_click(on_view_gallery_button_click)


    display(f)
    #display(update_gallery_button)
    # display(widgets.HBox([update_hovertext_button, clear_button]))
    #display(view_gallery_button)
    #display(view_gallery_output)
    
    # create uploader instance for the save/load callback functions with FileUpload()
    uploader = FileUpload(accept=".txt")
    
    
    # save gallery as a text file with button callback function    
    def save_as_text(_):
        global gallery 
        display(uploader)

        # def save_as_text_button_click(sender):
        #     filename = next(iter(uploader.value))
        #     basename, extension = os.path.splitext(filename)
        #     basename = basename.replace('_text', '')
        #     txt_filename = os.path.join('txt', f'{basename}_text.txt')
        #     try:
        #         with open(txt_filename, 'w', encoding='utf-8') as txt_file:
        #             for key, value in gallery.items():
        #                 txt_file.write(f"{key}: {value}\n")
        #         print(f"'gallery' dictionary saved as text to {txt_filename}")
        #     except Exception as e:
        #         print(f"Error saving gallery as text: {e}")
        
        def save_as_text_button_click(sender):
            filename = next(iter(uploader.value))
            basename, extension = os.path.splitext(filename)
            basename = basename.replace('_text', '')
            txt_filename = os.path.join('txt', f'{basename}_text.txt')
            try:
                with open(txt_filename, 'w', encoding='utf-8') as txt_file:
                    txt_file.write(f"gallery = {{\n")
                    for key, value in gallery.items():
                        txt_file.write(f"    '{key}': {value},\n")
                    txt_file.write(f"}}\n")
                print(f"'gallery' dictionary saved as text to {txt_filename}")
            except Exception as e:
                print(f"Error saving gallery as text: {e}")

        # save button generation
        save_as_text_button = widgets.Button(description="Save as Text")
        try: 
            save_as_text_button.on_click(save_as_text_button_click)
        except Exception as e: 
            print(f"No file selected from upload widget: {e}") 

        display(save_as_text_button)
        
    
    # save gallery button callback function
    def save_gallery(_):
        global gallery
        display(uploader)
        def save_gallery_button_click(sender):
            #nonlocal gallery
            filename = next(iter(uploader.value))
            try:
                # save as a pickled binary file
                with open(f'txt/bin/{filename}', 'wb') as bin_file: 
                    pickle.dump(gallery, bin_file)
                print(f"'gallery' dictionary saved to txt/bin/{filename}")
                
            # print exception if file does not save as bin and/or txt file    
            except Exception as e: 
                print(f"Error saving gallery: {e}")
                
        # save button generation
        save_gallery_button = widgets.Button(description="Save Gallery")
        try: 
            save_gallery_button.on_click(save_gallery_button_click)
        except Exception as e: 
            print(f"No file selected from Upload () widget: {e}")
        display(save_gallery_button)
    
    # load gallery button callback function 
    def load_gallery(_): 
        global gallery
        display(uploader)
        def load_gallery_button_click(sender): 
            global gallery
            #nonlocal gallery
            filename = next(iter(uploader.value))
            try:
                with open(f'txt/bin/{filename}', 'rb') as file: 
                    gallery = pickle.load(file, encoding='latin-1')
                print(f"'gallery' dictionary loaded from {filename}")
            except Exception as e:
                print(f"Error loading gallery: {e}")
                
        #load button generation
        load_gallery_button = widgets.Button(description="Load Gallery")
        try: 
            load_gallery_button.on_click(load_gallery_button_click)
        except Exception as e: 
                print(f"No file selected from Upload () widget: {e}")
        display(load_gallery_button)
                            
                                  
    # create on_click action (widget on_click executes/runs above functions)
    save_gallery_button = widgets.Button(description="Save File")
    load_gallery_button = widgets.Button(description="Open File")
    save_as_text_button = widgets.Button(description="Save as Text")
    
    save_gallery_button.on_click(save_gallery)
    load_gallery_button.on_click(load_gallery)
    save_as_text_button.on_click(save_as_text)
    
    # create horizontal box to hold the buttons & layout format
    buttons_horizontal_box = widgets.HBox([save_gallery_button, load_gallery_button, save_as_text_button])
    # buttons_horizontal_box.layout.justify_content = 'space-between'
    # buttons_horizontal_box.layout.align_items = 'center'
    
    # display horizontal button box
    display(buttons_horizontal_box) 
    
    #display(save_gallery_button, load_gallery_button, save_as_json_button)
    
    
    # button that clears all output
    
    clear_button = widgets.Button(description="Clear Output")
    
    def clear_button_on_click(clear_button): 
        clear_output(wait=True)
    
    clear_button.on_click(clear_button_on_click)
    
    display(widgets.HBox([update_hovertext_button, clear_button]))
    
submit_dim_button.on_click(on_submit_dim_button)

IntText(value=50, description='X:')

IntText(value=75, description='Y:')

Button(description='Submit X & Y dim', style=ButtonStyle())

FigureWidget({
    'data': [{'hovertext': [Point 0<br>X: 0<br>Y: 0, Point 1<br>X: 1<br>Y: 0,
                            Point 2<br>X: 2<br>Y: 0, ..., Point 3747<br>X: 47<br>Y:
                            74, Point 3748<br>X: 48<br>Y: 74, Point 3749<br>X:
                            49<br>Y: 74],
              'marker': {'color': [#B2B4B8, #B2B4B8, #B2B4B8, ..., #B2B4B8,
                                   #B2B4B8, #B2B4B8],
                         'size': [8, 8, 8, ..., 8, 8, 8]},
              'mode': 'markers',
              'type': 'scatter',
              'uid': 'cdbf655c-7052-4e57-9ac3-2243c30d4dfd',
              'x': array([ 0,  1,  2, ..., 47, 48, 49]),
              'y': array([ 0,  0,  0, ..., 74, 74, 74])}],
    'layout': {'height': 800,
               'hovermode': 'closest',
               'template': '...',
               'title': {'text': 'Venue Builder - Section Constructor'},
               'width': 800,
               'xaxis': {'autorange': True, 'range': [-25, 74], '

FileUpload(value={}, accept='.txt', description='Upload')

Button(description='Load Gallery', style=ButtonStyle())

'gallery' dictionary loaded from meridian_arts_centre_george_weston.txt
